# Training a BERT4Rec baseline  

In [ ]:
!pip install transformers4rec[pytorch,nvtabular]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 66.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 KB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3

In [ ]:
import os
import glob

import torch 
import transformers4rec.torch as tr

from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

In [ ]:
from merlin_standard_lib import Schema

# Define schema object to pass it to the TabularSequenceFeatures class
SCHEMA_PATH = '/content/drive/MyDrive/dataset_rees46/processed_nvt/schema.pbtxt'
schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(['product_id-list'])



In [ ]:
schema

[{'name': 'product_id-list', 'value_count': {'min': '2', 'max': '20'}, 'type': 'INT', 'int_domain': {'name': 'product_id', 'max': '166795', 'is_categorical': True}, 'annotation': {'tag': ['item', 'id', 'list', 'item_id', 'categorical'], 'comment': ['{"freq_threshold": 0.0, "start_index": 1.0, "is_list": true, "embedding_sizes": {"dimension": 512.0, "cardinality": 166796.0}, "cat_path": ".//categories/unique.product_id.parquet", "max_size": 0.0, "is_ragged": true, "num_buckets": null, "dtype_item_size": 64.0}']}}]

In [ ]:
!head -50 $SCHEMA_PATH

feature {
  name: "product_id-count"
  type: INT
  int_domain {
    name: "product_id"
    max: 166795
    is_categorical: true
  }
  annotation {
    tag: "categorical"
    extra_metadata {
      type_url: "type.googleapis.com/google.protobuf.Struct"
      value: "\n\034\n\017dtype_item_size\022\t\021\000\000\000\000\000\000@@\nG\n\017embedding_sizes\0224*2\n\026\n\tdimension\022\t\021\000\000\000\000\000\000\200@\n\030\n\013cardinality\022\t\021\000\000\000\000`\\\004A\n\033\n\016freq_threshold\022\t\021\000\000\000\000\000\000\000\000\n\017\n\tis_ragged\022\002 \000\n\r\n\007is_list\022\002 \000\n\021\n\013num_buckets\022\002\010\000\n\025\n\010max_size\022\t\021\000\000\000\000\000\000\000\000\n5\n\010cat_path\022)\032\'.//categories/unique.product_id.parquet\n\030\n\013start_index\022\t\021\000\000\000\000\000\000\360?"
    }
  }
}
feature {
  name: "user_session"
  type: INT
  int_domain {
    name: "user_session"
    max: 9244422
    is_categorical: true
  }
  annotation {
    t

In [ ]:
#Input 
sequence_length, d_model = 20, 320

# Define input module to process tabular input-features and to prepare masked inputs
inputs= tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=sequence_length,
    d_output=d_model,
    masking="mlm",
)

In [ ]:
inputs

TabularSequenceFeatures(
  (to_merge): ModuleDict(
    (categorical_module): SequenceEmbeddingFeatures(
      (filter_features): FilterFeatures()
      (embedding_tables): ModuleDict(
        (product_id-list): Embedding(166796, 64, padding_idx=0)
      )
    )
  )
  (_aggregation): ConcatFeatures()
  (projection_module): SequentialBlock(
    (0): DenseBlock(
      (0): Linear(in_features=64, out_features=320, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (_masking): MaskedLanguageModeling()
)

La tecnica di tying embedding è spesso usata in NLP e "collega" i pesi dell'input embedding con la matrice dell'output projection layer. 
Questo viene fatto perchè l'input e l'output dei modelli sono nello stesso spazio, cioè sono delle parole (in NLP) e dovrebbero ricadere nello stesso spazio vettoriale. 
I modelli di rec sys in modo analogo hanno gli item id come input e come output. 

si ridurrebbero i parametri del modello, poichè il numero di embeddings delle features categoriche ad alta cardinalità sono molto maggiori rispetto a NLP. 
Un altro beneficio è che si ha una regolarizzazione del modello, adattando la sua complessità ai dati.

I modelli rec sys di deep learing sono generalmente limitati dallo spazio di memoria disponibile durante il training e l'evaluation (sono memory-bound) e molti parametri sono concentrati in grandi tabelle di embedding. Qui subentra la tecnica di tying embedding che riduce i requisiti di memoria utilizzando soltanto una matrice di proiezione sia per gli item sia per le rappresentazioni dell'output. 
Questa riduzione dei parametri non è il solo motivo per cui si utilizza la tecnica di tying embedding. 
Questa tecnica introduce inoltre una operazione di fattorizzazione della matrice tra gli item embeddings e la rappresentazione finale della sessione. 



In [ ]:
#import transformers4rec.config.transformer as hf

transformer_config = tr.AlbertConfig.build(
    d_model=d_model, 
    item_embedding_dim = 320,
    n_head=8, 
    n_layer=2, 
    total_seq_length=sequence_length, 
    stochastic_shared_embeddings_replacement_prob = 0.06, #regularization
    input_dropout = 0.1,
    dropout = 0.0, #regularization
    label_smoothing = 0.2, #regularization (proved to be useful in train/val accuracy)
    weight_decay = 9.565968888623912e-05, #regularization,
    item_id_embeddings_init_std = 0.11,
    mlm_probability = 0.6,
    eval_on_last_item_seq_only = True,
    mf_constrained_embeddings = True,
    layer_norm_featurewise = True,
    num_hidden_groups = 1,
    inner_group_num = 1
)

# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs,
    tr.MLPBlock([d_model]),
    tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head for to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True,
                              metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  
                                       RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)

# Get the end-to-end Model class 
model = tr.Model(head)

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
from transformers4rec.torch.utils.data_utils import MerlinDataLoader

#Set arguments for training 
training_args = T4RecTrainingArguments(
            output_dir="/content/drive/MyDrive/dataset_rees46/bert",
            max_sequence_length=20,
            data_loader_engine='merlin',
            num_train_epochs=10, 
            dataloader_drop_last=True,
            compute_metrics_each_n_steps = 1,
            per_device_train_batch_size = 192,
            per_device_eval_batch_size = 512,
            gradient_accumulation_steps = 1,
            learning_rate=0.0004904752786458524,
            report_to = [],
            logging_steps=200,
        )

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True,
)

In [ ]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/drive/MyDrive/dataset_rees46/sessions_by_day")

In [ ]:
%%time
start_time_window_index = 1
final_time_window_index = 31
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/test.parquet"))
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset_or_path = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    # Evaluate on the following day
    trainer.eval_dataset_or_path = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()

********************
Launch training for day 1 are:
********************



/usr/local/lib/python3.8/dist-packages/merlin/io/dataset.py:253: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 111936
  Num Epochs = 10
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 5830


Step,Training Loss
200,10.444900
400,9.120000
600,8.929000
800,8.661400
1000,8.598300
1200,8.467800
1400,8.353100
1600,8.291800
1800,8.197000
2000,8.093600


Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/

********************
Eval results for day 2 are:	

********************

 eval_/loss = 8.172948837280273
 eval_/next-item/ndcg_at_10 = 0.0967572033405304
 eval_/next-item/ndcg_at_20 = 0.11630704253911972
 eval_/next-item/recall_at_10 = 0.18117186427116394
 eval_/next-item/recall_at_20 = 0.25898435711860657
 eval_runtime = 0.7703
 eval_samples_per_second = 16617.182
 eval_steps_per_second = 32.455
********************
Launch training for day 2 are:
********************



...

********************
Eval results for day 29 are:	

********************

 eval_/loss = 6.384833812713623
 eval_/next-item/ndcg_at_10 = 0.20521201193332672
 eval_/next-item/ndcg_at_20 = 0.23246820271015167
 eval_/next-item/recall_at_10 = 0.3602343797683716
 eval_/next-item/recall_at_20 = 0.4678124785423279
 eval_runtime = 0.7791
 eval_samples_per_second = 16428.666
 eval_steps_per_second = 32.087
********************
Launch training for day 29 are:
********************



***** Running training *****
  Num examples = 107328
  Num Epochs = 10
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 5590


Step,Training Loss
200,6.094200
400,6.075500
600,6.039500
800,5.981500
1000,5.954200
1200,5.899000
1400,5.867100
1600,5.877000
1800,5.819700
2000,5.833000


Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/

********************
Eval results for day 30 are:	

********************

 eval_/loss = 6.447874546051025
 eval_/next-item/ndcg_at_10 = 0.20332306623458862
 eval_/next-item/ndcg_at_20 = 0.23116347193717957
 eval_/next-item/recall_at_10 = 0.35789060592651367
 eval_/next-item/recall_at_20 = 0.4682031273841858
 eval_runtime = 0.7713
 eval_samples_per_second = 16595.475
 eval_steps_per_second = 32.413
CPU times: user 2h 34min 7s, sys: 1min 43s, total: 2h 35min 50s
Wall time: 1h 17min 15s


In [ ]:
print("BERT results:")
for key, value in  model.compute_metrics().items(): 
  print('%s: %s' % (key, value.item()))

BERT results:
next-item/ndcg_at_10: 0.20332306623458862
next-item/ndcg_at_20: 0.23116347193717957
next-item/recall_at_10: 0.35789060592651367
next-item/recall_at_20: 0.4682031273841858


In [ ]:
with open("/content/drive/MyDrive/dataset_rees46/results.txt", 'w') as f:
    f.write('\n')
    f.write('Albert - MLM:')
    f.write('\n')
    for key, value in  model.compute_metrics().items(): 
        f.write('%s %s\n' % (key, value.item()))